# Predictive model: Business Use Case

## Goal

- Using the available [Road Safety Data](https://www.data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data) create a predictive modeling business use case
- Desired use case: **On-premise forecast of *dangarous* traffic situations**
- Train model capable to predict accident severity "score": Overall accident severity

## Considerations

- Take all available information into account including spatiotemporal, environmental, vehicle
- Important when defining features: 
    - Only features available *before* accident can be used
    - Do not utilize features not to be used in practice, e.g. 
        - due to GDPR (driver properties)
        - busincess-strategic (car model name)

In [ ]:
import os
import pandas
import mlflow
import numpy as np
from pathlib import Path

from helpers.utils import (
    load_yaml,
    infer_catboost_feature_types
)

from sklearn import set_config
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix
from sklearn.model_selection import ShuffleSplit

from catboost import CatBoostRegressor, Pool

set_config(transform_output="pandas")
assets_dir = Path(os.environ["DATA_DIR"]) / "assets"

In [ ]:
X = pandas.read_pickle(assets_dir / "accidents_vehicles_casualties_dataset_result.pkl")
X = X.query("`accident.year` == 2021")
y = X.pop("target")

## Training and logging

- Define model parameters
- Apply training rounds and log params and metrics to tracking server
- Log trained model as model artifact

In [ ]:
feat = infer_catboost_feature_types(X)

catboost_init_params = {
    "cat_features": feat["categorical"], 
    "text_features": feat["text"], 
    "od_type": "Iter", 
    "iterations": 400,
    "train_dir": "/tmp/catboost"
}

catboost_fit_params = {
    "early_stopping_rounds": 101,
    "verbose": 100
}

model = CatBoostRegressor(**catboost_init_params)

In [ ]:
mlflow.set_registry_uri(os.environ["MLFLOW_TRACKING_URI"])
mlflow.set_experiment("casualty-regression")

In [ ]:
with mlflow.start_run() as run:
    
    run_id = run.info.run_id
    
    mlflow.log_params(catboost_init_params)
    mlflow.log_params(catboost_fit_params)
    mlflow.log_param("n_accidents", len(X))

    splitter = ShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    
    for train, test in splitter.split(X):
        Xtrain = X.iloc[train]
        ytrain = y.iloc[train]

        data_test = Pool(
            X.iloc[test], 
            y.iloc[test], 
            text_features=feat["text"],
            cat_features=feat["categorical"]
        )

        train, val = next(
            ShuffleSplit(n_splits=1, test_size=0.05, random_state=42)
            .split(Xtrain)
        )

        data_val = Pool(
            X.iloc[val], 
            y.iloc[val], 
            text_features=feat["text"],
            cat_features=feat["categorical"]
        )

        data_train = Pool(
            X.iloc[train], 
            y.iloc[train], 
            text_features=feat["text"],
            cat_features=feat["categorical"]
        )

        model.fit(data_train, eval_set=data_val, **catboost_fit_params)

        ypred = model.predict(data_test)
        
        mae = mean_absolute_error(data_test.get_label(), ypred)
        mse = mean_squared_error(data_test.get_label(), ypred)
        
        mlflow.log_metrics(
            {
                "mae": mae,
                "mse": mse
            }
        )

        # Confusion matrix
        confusion = pandas.DataFrame(
            confusion_matrix(
                data_test.get_label(), 
                np.round(ypred, 0).astype(int)
            )
        )
        confusion.to_csv("/tmp/confusion.csv")
        mlflow.log_artifact("/tmp/confusion.csv")

In [ ]:
with mlflow.start_run(run_id=run_id):
    # Log model artifact
    model.fit(X, y, **catboost_fit_params)
    mlflow.catboost.log_model(model, artifact_path="model")